In [ ]:
##BBC

In [ ]:
import time
import re
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
edge_options.add_argument("--headless")
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

url = "https://www.bbc.co.uk/sport/football/teams/liverpool"
driver.get(url)

time.sleep(5)
html = driver.page_source
driver.quit()
soup = BeautifulSoup(html, "html.parser")
articles = soup.find_all("article", class_="ssrcss-1epmku9-ContentPost e6wdqbx1")

news_data = []

for article in articles:
    date_span = article.find("span")
    if date_span:
        raw_date = date_span.get_text(separator=" ", strip=True)
        
        time_matches = re.findall(r"(\d{1,2}:\d{2})", raw_date)
        if time_matches:
            time_text = time_matches[0]
        else:
            time_text = "00:00"
        
        date_part = re.sub(r"\d{1,2}:\d{2}", "", raw_date).strip()
        tokens = date_part.split()
        if len(tokens) >= 2:
            extracted_date_str = " ".join(tokens[:2])
            try:
                date_obj = datetime.strptime(extracted_date_str, "%d %B")
                date_text = date_obj.replace(year=2025).strftime("%#m/%#d/%Y")
            except Exception as e:
                date_text = "3/17/2025"
        else:
            date_text = "3/17/2025"
    else:
        date_text = "3/17/2025"
        time_text = "00:00"
    
    headline_tag = article.find("span", class_="ssrcss-189b1h2-HeadlineWrap e14e9ror2")
    if headline_tag:
        headline_text = headline_tag.get_text(strip=True)
    else:
        headline_text = "No Headline"
    
    paragraphs = article.find_all("p", class_=re.compile("Paragraph"))
    if paragraphs:
        body_text = " ".join(p.get_text(strip=True) for p in paragraphs)
    else:
        body_text = "No Content"
    
    news_data.append([date_text, time_text, headline_text, body_text])

df = pd.DataFrame(news_data, columns=["Date", "Time", "Headline", "Body Context"])
print(df.head())
df.to_csv("Test.csv", index=False)

        Date   Time                                           Headline  \
0  3/17/2025  15:14  Is Salah's cup finals record his Achilles heel...   
1  3/17/2025  15:00  Liverpool 'can suffer' final defeat - Jamespub...   
2  3/17/2025  13:55  'We want to bounce back, and we will'published...   
3  3/17/2025  12:06  'Lack of quality depth must be addressed'publi...   
4  3/17/2025  09:55  'We played into their hands'published at 09:55...   

                                        Body Context  
0  Alongside his team-mates, it was a difficult d...  
1  Former Liverpool goalkeeper David James says L...  
2  Liverpool captain Virgil van Dijk on Instagram...  
3  In the space of a week, the Arne Slot machine ...  
4  Andy Robertson says Liverpool "didn't use the ...  


In [ ]:
import time
import re
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

edge_driver_path = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"
edge_options = EdgeOptions()
edge_options.add_argument("--headless")
service = EdgeService(edge_driver_path)
driver = webdriver.Edge(service=service, options=edge_options)

all_news_data = [] 

for page in range(1, 41):
    if page == 1:
        url = "https://www.bbc.co.uk/sport/football/teams/liverpool"
    else:
        url = f"https://www.bbc.co.uk/sport/football/teams/liverpool?page={page}"

    print(f"Scraping Page {page}: {url}")
    
    driver.get(url)
    time.sleep(3)
    html = driver.page_source

    soup = BeautifulSoup(html, "html.parser")
    articles = soup.find_all("article", class_="ssrcss-1epmku9-ContentPost e6wdqbx1")

    for article in articles:
        date_span = article.find("span")
        if date_span:
            raw_date = date_span.get_text(separator=" ", strip=True)
            
            time_matches = re.findall(r"(\d{1,2}:\d{2})", raw_date)
            if time_matches:
                time_text = time_matches[0]
            else:
                time_text = "00:00"
            
            date_part = re.sub(r"\d{1,2}:\d{2}", "", raw_date).strip()
            tokens = date_part.split()
            if len(tokens) >= 2:
                extracted_date_str = " ".join(tokens[:2])
                try:
                    date_obj = datetime.strptime(extracted_date_str, "%d %B")
                    date_text = date_obj.replace(year=2025).strftime("%#m/%#d/%Y")
                except Exception:
                    date_text = "3/17/2025"
            else:
                date_text = "3/17/2025"
        else:
            date_text = "3/17/2025"
            time_text = "00:00"
        
        headline_tag = article.find("span", class_="ssrcss-189b1h2-HeadlineWrap e14e9ror2")
        if headline_tag:
            headline_text = headline_tag.get_text(strip=True)
            lower_headline = headline_text.lower()
            idx = lower_headline.find("published at")
            if idx != -1:
                headline_text = headline_text[:idx].strip()
        else:
            headline_text = "No Headline"
        
        paragraphs = article.find_all("p", class_=re.compile("Paragraph"))
        if paragraphs:
            body_text = " ".join(p.get_text(strip=True) for p in paragraphs)
        else:
            body_text = "No Content"
        
        all_news_data.append([date_text, time_text, headline_text, body_text])

driver.quit()
df = pd.DataFrame(all_news_data, columns=["Date", "Time", "Headline", "Body Context"])
df.to_csv("BBC_AllPages.csv", index=False)
print("Scraping complete! Data saved to AllPages.csv")

Scraping Page 1: https://www.bbc.co.uk/sport/football/teams/liverpool
Scraping Page 2: https://www.bbc.co.uk/sport/football/teams/liverpool?page=2
Scraping Page 3: https://www.bbc.co.uk/sport/football/teams/liverpool?page=3
Scraping Page 4: https://www.bbc.co.uk/sport/football/teams/liverpool?page=4
Scraping Page 5: https://www.bbc.co.uk/sport/football/teams/liverpool?page=5
Scraping Page 6: https://www.bbc.co.uk/sport/football/teams/liverpool?page=6
Scraping Page 7: https://www.bbc.co.uk/sport/football/teams/liverpool?page=7
Scraping Page 8: https://www.bbc.co.uk/sport/football/teams/liverpool?page=8
Scraping Page 9: https://www.bbc.co.uk/sport/football/teams/liverpool?page=9
Scraping Page 10: https://www.bbc.co.uk/sport/football/teams/liverpool?page=10
Scraping Page 11: https://www.bbc.co.uk/sport/football/teams/liverpool?page=11
Scraping Page 12: https://www.bbc.co.uk/sport/football/teams/liverpool?page=12
Scraping Page 13: https://www.bbc.co.uk/sport/football/teams/liverpool?page=1